In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import os
import re
import time
import torch
import torch.nn.functional as F

from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from glob import glob, iglob

In [2]:
def conv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=True):
    '''
    Creates a convolutional layer, with optional batch normalization.
    '''
    layers = []
    conv_layer = nn.Conv2d(in_channels, out_channels,
                           kernel_size, stride, padding, bias=False)

    # append conv layer
    layers.append(conv_layer)

    if batch_norm:
        # append batchnorm layer
        layers.append(nn.BatchNorm2d(out_channels))

    # using Sequential container
    return nn.Sequential(*layers)


# helper deconv function
def deconv(in_channels, out_channels, kernel_size, stride=2, padding=1, batch_norm=True):
    '''
    Creates a transposed-convolutional layer, with optional batch normalization.
    '''
    # create a sequence of transpose + optional batch norm layers
    layers = []
    transpose_conv_layer = nn.ConvTranspose2d(in_channels, out_channels,
                                              kernel_size, stride, padding, bias=False)
    # append transpose convolutional layer
    layers.append(transpose_conv_layer)

    if batch_norm:
        # append batchnorm layer
        layers.append(nn.BatchNorm2d(out_channels))

    return nn.Sequential(*layers)

In [18]:
image = torch.ones(1, 3,69,69)
image.shape

torch.Size([1, 3, 69, 69])

In [19]:
conv_dim = 4
# 69x69 input
# first layer, no batch_norm
conv1 = conv(3, conv_dim, 5, padding=0, stride=4, batch_norm=False)
# 17x17 out
conv2 = conv(conv_dim, conv_dim*2, 3, padding=0)
# 8x8 out
conv3 = conv(conv_dim*2, conv_dim*4, 4, padding=0)
# 3x3 out

# final, fully-connected layer
fc = nn.Linear(conv_dim*4*3*3, 1)

In [20]:
image = conv1(image)
image.shape

torch.Size([1, 4, 17, 17])

In [21]:
image = conv2(image)
image.shape

torch.Size([1, 8, 8, 8])

In [22]:
image = conv3(image)
image.shape

torch.Size([1, 16, 3, 3])

In [31]:
# first, fully-connected layer
z_size = 64
tfc = nn.Linear(z_size, conv_dim*4*3*3)

# transpose conv layers
t_conv1 = deconv(conv_dim*4, conv_dim*2, 4, padding=0)
t_conv2 = deconv(conv_dim*2, conv_dim, 3, padding=0)
t_conv3 = deconv(conv_dim, 3, 5, padding=0, stride=4, batch_norm=False)

In [32]:
latent = torch.ones(1, z_size)
latent.shape

torch.Size([1, 64])

In [33]:
latent = tfc(latent)
latent.shape

torch.Size([1, 144])

In [34]:
latent = latent.view(-1, conv_dim*4, 3, 3)
latent = t_conv1(latent)
latent.shape

torch.Size([1, 8, 8, 8])

In [35]:
latent = t_conv2(latent)
latent.shape

torch.Size([1, 4, 17, 17])

In [36]:
latent = t_conv3(latent)
latent.shape

torch.Size([1, 3, 69, 69])